## Инструкции к решению задачи

Задание состоит из трёх частей. Если не сказано обратное, то уровень значимости нужно принять равным 0.05.

### Часть 1: применение t-критерия Стьюдента
В первой части вам нужно будет применить критерий Стьюдента для проверки гипотезы о равенстве средних в двух независимых выборках. Применить критерий для каждого гена нужно будет дважды:

для групп normal (control) и early neoplasia (treatment)

для групп early neoplasia (control) и cancer (treatment)

В качестве ответа в этой части задания необходимо указать количество статистически значимых отличий, которые вы нашли с помощью t-критерия Стьюдента, то есть число генов, у которых p-value этого теста оказался меньше, чем уровень значимости.



In [3]:
import pandas as pd
from scipy.stats import ttest_ind

In [17]:
data.Diagnosis.unique()

array(['normal', 'early neoplasia', 'cancer'], dtype=object)

In [2]:
data = pd.read_csv('gene_high_throughput_sequencing.csv')
data.head()

,Patient_id,Diagnosis,LOC643837,LOC100130417,SAMD11,NOC2L,KLHL17,PLEKHN1,C1orf170,HES4,...,CLIC2,RPS4Y1,ZFY,PRKY,USP9Y,DDX3Y,CD24,CYorf15B,KDM5D,EIF1AY
0,STT5425_Breast_001_normal,normal,1.257614,2.408148,13.368622,9.494779,20.880435,12.722017,9.494779,54.349694,...,4.761250,1.257614,1.257614,1.257614,1.257614,1.257614,23.268694,1.257614,1.257614,1.257614
1,STT5427_Breast_023_normal,normal,4.567931,16.602734,42.477752,25.562376,23.221137,11.622386,14.330573,72.445474,...,6.871902,1.815112,1.815112,1.815112,1.815112,1.815112,10.427023,1.815112,1.815112,1.815112
2,STT5430_Breast_002_normal,normal,2.077597,3.978294,12.863214,13.728915,14.543176,14.141907,6.232790,57.011005,...,7.096343,2.077597,2.077597,2.077597,2.077597,2.077597,22.344226,2.077597,2.077597,2.077597
3,STT5439_Breast_003_normal,normal,2.066576,8.520713,14.466035,7.823932,8.520713,2.066576,10.870009,53.292034,...,5.200770,2.066576,2.066576,2.066576,2.066576,2.066576,49.295538,2.066576,2.066576,2.066576
4,STT5441_Breast_004_normal,normal,2.613616,3.434965,12.682222,10.543189,26.688686,12.484822,1.364917,67.140393,...,11.227770,1.364917,1.364917,1.364917,1.364917,1.364917,23.627911,1.364917,1.364917,1.364917


In [27]:
student = pd.DataFrame(columns = ['control', 'treatment', 'gen', 'stat', 'pvalue'])
i = 0
for column in list(data.columns.drop(['Patient_id', 'Diagnosis'])):
    stat, pvalue = ttest_ind(data[column].loc[data[data.Diagnosis == 'normal'].index],
                             data[column].loc[data[data.Diagnosis == 'early neoplasia'].index], equal_var = False)
    student.loc[i] = ['normal', 'early neoplasia', column, stat, pvalue]
    i += 1

    stat, pvalue = ttest_ind(data[column].loc[data[data.Diagnosis == 'early neoplasia'].index],
                             data[column].loc[data[data.Diagnosis == 'cancer'].index], equal_var = False)
    student.loc[i] = ['early neoplasia', 'cancer',column, stat, pvalue]
    i += 1

student

,control,treatment,gen,stat,pvalue
0,normal,early neoplasia,LOC643837,0.400289,0.690766
1,early neoplasia,cancer,LOC643837,0.824849,0.413735
2,normal,early neoplasia,LOC100130417,-4.608766,0.000032
3,early neoplasia,cancer,LOC100130417,0.452236,0.653429
4,normal,early neoplasia,SAMD11,-1.929277,0.060273
...,...,...,...,...,...
31491,early neoplasia,cancer,CYorf15B,-0.613696,0.542939
31492,normal,early neoplasia,KDM5D,-0.262910,0.793925
31493,early neoplasia,cancer,KDM5D,-0.579209,0.565753
31494,normal,early neoplasia,EIF1AY,-0.441420,0.661031


In [113]:
p1 = len(student.loc[(student.control == 'normal') & (student.pvalue < 0.05)])
p2 = len(student.loc[(student.control == 'early neoplasia') & (student.pvalue < 0.05)])
p1, p2

(1575, 3490)

In [114]:
def write_answer(task, *answer):
    for i in range(len(answer)):
        with open("answer" + str(task) + "_" + str(i+1) +".txt", "w") as fout:
            fout.write(str(answer[i]))

In [88]:
write_answer(1, p1, p2)

### Часть 2: поправка методом Холма

Для этой части задания вам понадобится модуль multitest из statsmodels.

В этой части задания нужно будет применить поправку Холма для получившихся двух наборов достигаемых уровней значимости из предыдущей части. Обратите внимание, что поскольку вы будете делать поправку для каждого из двух наборов p-value отдельно, то проблема, связанная с множественной проверкой останется.

Для того, чтобы ее устранить, достаточно воспользоваться поправкой Бонферрони, то есть использовать уровень значимости 0.05 / 2 вместо 0.05 для дальнейшего уточнения значений p-value c помощью метода Холма.

В качестве ответа к этому заданию требуется ввести количество значимых отличий в каждой группе после того, как произведена коррекция Холма-Бонферрони. Причем это число нужно ввести с учетом практической значимости: посчитайте для каждого значимого изменения fold change и выпишите в ответ число таких значимых изменений, абсолютное значение fold change которых больше, чем $1.5$.

#### Обратите внимание, что

применять поправку на множественную проверку нужно ко всем значениям достигаемых уровней значимости, а не только для тех, которые меньше значения уровня доверия.

при использовании поправки на уровне значимости 0.025 меняются значения достигаемого уровня значимости, но не меняется значение уровня доверия (то есть для отбора значимых изменений скорректированные значения уровня значимости нужно сравнивать с порогом 0.025, а не 0.05)!


In [54]:
from statsmodels.stats.multitest import multipletests

In [115]:
normal_data = student.loc[student.control == 'normal']
early_data = student.loc[student.control == 'early neoplasia']

In [116]:
reject_n, p_corrected_n, a1, a2 = multipletests(normal_data.pvalue, alpha = 0.025, method = 'holm')
reject_e, p_corrected_e, a1, a2 = multipletests(early_data.pvalue, alpha = 0.025, method = 'holm')

In [117]:
normal_data['p_correct'] = p_corrected_n
normal_data['reject'] = reject_n
early_data['p_correct'] = p_corrected_e
early_data['reject'] = reject_e

<ipython-input-117-133414d12618>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  normal_data['p_correct'] = p_corrected_n
<ipython-input-117-133414d12618>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  normal_data['reject'] = reject_n
<ipython-input-117-133414d12618>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_g

In [118]:
normal_data.loc[normal_data.p_correct < 0.025]

,control,treatment,gen,stat,pvalue,p_correct,reject
14488,normal,early neoplasia,PCSK4,-5.806801,7.955435e-07,0.012527,True
19640,normal,early neoplasia,EEF1A2,-6.524922,8.498742e-08,0.001338,True


In [119]:
early_data.loc[early_data.p_correct < 0.025]

,control,treatment,gen,stat,pvalue,p_correct,reject
95,early neoplasia,cancer,GABRD,-6.583440,2.276043e-07,0.003575,True
567,early neoplasia,cancer,EXTL1,6.600818,1.087046e-07,0.001709,True
633,early neoplasia,cancer,CD164L2,5.826854,1.002085e-06,0.015708,True
2211,early neoplasia,cancer,NES,6.709055,1.041679e-07,0.001637,True
2477,early neoplasia,cancer,FMO2,5.751625,1.103311e-06,0.017292,True
...,...,...,...,...,...,...,...
30181,early neoplasia,cancer,LAMC3,8.424379,1.938708e-09,0.000031,True
30899,early neoplasia,cancer,SLC7A3,7.046264,3.981097e-08,0.000626,True
31153,early neoplasia,cancer,CAPN6,6.524854,7.554205e-08,0.001188,True
31281,early neoplasia,cancer,GPC3,5.805582,5.832350e-07,0.009152,True


In [120]:
list_gen_n = list(normal_data.gen.loc[normal_data.p_correct < 0.025])
list_gen_e = list(early_data.gen.loc[early_data.p_correct < 0.025])

In [121]:
def change_fold(gens, dataset, statement):
    count_cf = 0
    for gen in gens:
        if statement == 1:
            mean_1 = dataset[gen].loc[dataset.Diagnosis == 'normal'].mean()
            mean_2 = dataset[gen].loc[dataset.Diagnosis == 'early neoplasia'].mean()
        else:
            mean_1 = dataset[gen].loc[dataset.Diagnosis == 'early neoplasia'].mean()
            mean_2 = dataset[gen].loc[dataset.Diagnosis == 'cancer'].mean()
        if mean_1 > mean_2:
            if abs(mean_1 / mean_2) > 1.5:
                count_cf += 1
        else:
            if abs(mean_2 / mean_1) > 1.5:
                count_cf += 1
        #print(mean_1, mean_2, mean_3)
    return count_cf
            
count_1 = change_fold(list_gen_n, data, 1)
count_2 = change_fold(list_gen_e, data, 2)
print(count_1, count_2)

2 77


In [122]:
write_answer(2, count_1, count_2)

### Часть 3: поправка методом Бенджамини-Хохберга
Данная часть задания аналогична второй части за исключением того, что нужно будет использовать метод Бенджамини-Хохберга.

Обратите внимание, что методы коррекции, которые контролируют FDR, допускает больше ошибок первого рода и имеют большую мощность, чем методы, контролирующие FWER. Большая мощность означает, что эти методы будут совершать меньше ошибок второго рода (то есть будут лучше улавливать отклонения от $H_0$, когда они есть, и будут чаще отклонять $H_0$, когда отличий нет).

В качестве ответа к этому заданию требуется ввести количество значимых отличий в каждой группе после того, как произведена коррекция Бенджамини-Хохберга, причем так же, как и во второй части, считать только такие отличия, у которых abs(fold change) > 1.5.

In [123]:
reject_n, p_corrected_n, a1, a2 = multipletests(normal_data.pvalue, alpha = 0.025, method = 'fdr_bh')
reject_e, p_corrected_e, a1, a2 = multipletests(early_data.pvalue, alpha = 0.025, method = 'fdr_bh')

In [124]:
normal_data['p_correct_fdr'] = p_corrected_n
normal_data['reject_fdr'] = reject_n
early_data['p_correct_fdr'] = p_corrected_e
early_data['reject_fdr'] = reject_e

<ipython-input-124-5f520b77f39c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  normal_data['p_correct_fdr'] = p_corrected_n
<ipython-input-124-5f520b77f39c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  normal_data['reject_fdr'] = reject_n
<ipython-input-124-5f520b77f39c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stabl

In [125]:
list_gen_n = list(normal_data.gen.loc[normal_data.p_correct_fdr < 0.025])

In [126]:
list_gen_e = list(early_data.gen.loc[early_data.p_correct_fdr < 0.025])

In [127]:
count_1 = change_fold(list_gen_n, data, 1)
count_2 = change_fold(list_gen_e, data, 2)
print(count_1, count_2)

4 524


In [128]:
write_answer(3, count_1, count_2)